In [1]:
import time
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf 

In [ ]:
# загрузка train, public_test, privat_test и тензора весов модели
!gdown 1hp4ln3vHYaBkeVquHdgu_QA5JZkQWFTP
!gdown 1SSzUoJI6AsS1mUFCp-Zdb0s13ZZJMNDl
!gdown 1gjQApadBmv4YjuZgv-L7FwpUyRV4LchO
!gdown 1TgoNl5WkRockrJXZGdJWTbb4Ibqq8at-

In [2]:
# загружаем входные данные !пропишите правильные пути файлов
train=pd.read_csv('/content/drive/MyDrive/RuCode6_2/train.csv')
test=pd.read_csv('/content/drive/MyDrive/RuCode6_2/public_test.csv')
testp=pd.read_csv('/content/drive/MyDrive/RuCode6_2/private_test.csv')

In [3]:
# создаём словарь символов
voc=list(CountVectorizer(lowercase=False, token_pattern= r"(?u)\b\w\w+\b", ngram_range=(1, 1), 
                         analyzer="char").fit(train['corrupted_text'].tolist()+train['correct_text'].tolist()).vocabulary_.keys())

In [4]:
# задаём слой посимвольной текстовой векторизации
tv=tf.keras.layers.TextVectorization(max_tokens=len(voc)+2, 
                                  standardize=None, split='character', 
                                  output_mode='int', output_sequence_length=59, pad_to_max_tokens=True, vocabulary=voc)

In [5]:
# кодируем target в sparse
y=tv(tf.constant(train['correct_text'].values)).numpy()

In [6]:
X_train,X_val,y_train,y_val=train_test_split(train[['corrupted_text']],
                                             y,
                                             test_size=0.1,
                                             random_state=0)

In [7]:
# модель: гибридная нейронная сеть из двух веток: резидуальная двунаправленная LSTM и свёрточная (по архитектуре близкая к U-net)
# конкатенируются и обрабатываются верхним слоем однонаправленной LSTM

inp=tf.keras.Input(shape=(1),dtype='string',name='inp') 

lay1__0=tv(inp)

lay2_0__0=tf.one_hot(lay1__0,depth=250)
lay2_1__0=tf.roll(lay2_0__0, shift=-1, axis=-2)
lay2_2__0=tf.roll(lay2_0__0, shift=1, axis=-2)

lay2__0=tf.keras.layers.Concatenate()([lay2_0__0,lay2_1__0,lay2_2__0])


lay3_1__0=tf.keras.layers.LSTM(256,kernel_initializer='glorot_uniform', return_sequences=True,go_backwards=False)(lay2__0)
lay3_2__0=tf.keras.layers.LSTM(256,kernel_initializer='glorot_uniform',return_sequences=True,go_backwards=True)(lay2__0)
lay3__0=tf.keras.layers.Concatenate()([lay3_1__0,lay3_2__0,lay2__0])

lay4_1__0=tf.keras.layers.LSTM(1024,kernel_initializer='glorot_uniform',return_sequences=True,go_backwards=False)(lay3__0)
lay4_2__0=tf.keras.layers.LSTM(1024,kernel_initializer='glorot_uniform',return_sequences=True,go_backwards=True)(lay3__0)


lay1__1=tf.keras.layers.TextVectorization(max_tokens=len(voc)+2, 
                                  standardize=None, split='character', 
                                  output_mode='int', output_sequence_length=64, pad_to_max_tokens=True, vocabulary=voc)(inp)
lay2__1=tf.one_hot(lay1__1,depth=250)
lay3__1=tf.keras.layers.Conv1D(256, 3, padding='same', activation='tanh')(lay2__1)

lay4__1=tf.keras.layers.MaxPool1D(2)(lay3__1)

lay5__1=tf.keras.layers.Conv1D(256, 3, padding='same', activation='tanh')(lay4__1)

lay6__1=tf.keras.layers.MaxPool1D(2)(lay5__1)

lay7__1=tf.keras.layers.Conv1D(256, 3, padding='same', activation='tanh')(lay6__1)

lay8__1=tf.keras.layers.MaxPool1D(2)(lay7__1)

lay9__1=tf.keras.layers.Flatten()(lay8__1)

lay11__1=tf.keras.layers.Dense(256*8, activation='tanh')(lay9__1)

lay12__1=tf.keras.layers.Reshape((8, 256))(lay11__1)
lay12_1__1=tf.keras.layers.Concatenate()([lay12__1,lay8__1])

lay13__1=tf.keras.layers.Conv1DTranspose(256, 2, strides=2, activation='tanh')(lay12_1__1)
lay13_1__1=tf.keras.layers.Concatenate()([lay13__1,lay6__1])

lay14__1=tf.keras.layers.Conv1DTranspose(256, 2, strides=2, activation='tanh')(lay13_1__1)
lay14_1__1=tf.keras.layers.Concatenate()([lay14__1,lay4__1])

lay15__1=tf.keras.layers.Conv1DTranspose(250, 2, strides=2, activation='tanh')(lay14_1__1)

lay16__1=lay15__1[:,:59,:]


lay4__0=tf.keras.layers.Concatenate()([lay4_1__0,lay4_2__0,lay3__0,lay2__0,lay16__1])

lay5=tf.keras.layers.LSTM(250,kernel_initializer='glorot_uniform', 
                          activation='softmax', return_sequences=True)(lay4__0)

model = tf.keras.Model(inputs=inp,outputs=lay5)
optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001)
model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001), 
              loss="sparse_categorical_crossentropy", 
              metrics="sparse_categorical_accuracy")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inp (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization_1 (TextVect  (None, 64)          0           ['inp[0][0]']                    
 orization)                                                                                       
                                                                                                  
 tf.one_hot_1 (TFOpLambda)      (None, 64, 250)      0           ['text_vectorization_1[0][0]']   
                                                                                                  
 conv1d (Conv1D)                (None, 64, 256)      192256      ['tf.one_hot_1[0][0]']       

In [ ]:
# предварительное обучение, для оценки точности и кривой обучения на отложенной выборке
history=model.fit(X_train[['corrupted_text']].astype('str').values, 
                  y_train, batch_size=1024,epochs=10, 
        validation_data=(X_val[['corrupted_text']].astype('str').values,
                         y_val))

In [ ]:
# дообучение на всех данных
history=model.fit(train[['corrupted_text']].astype('str').values, 
                  y, batch_size=1024,epochs=40)

In [ ]:
# предварительное предсказание на паблик тесте
ansd1=np.argmax(model.predict(test[['corrupted_text']].astype('str').values[:50000],batch_size=1024),axis=-1)
ansd2=np.argmax(model.predict(test[['corrupted_text']].astype('str').values[50000:100000],batch_size=1024),axis=-1)
ansd3=np.argmax(model.predict(test[['corrupted_text']].astype('str').values[100000:150000],batch_size=1024),axis=-1)
ansd4=np.argmax(model.predict(test[['corrupted_text']].astype('str').values[150000:],batch_size=1024),axis=-1)
ansl=[]
for cnt in np.vstack((ansd1,ansd2,ansd3,ansd4)):
  ansl.append(''.join(list(map(lambda x: tvvoc[x], cnt.tolist()))))
test['correct_text']=ansl

In [ ]:
# оценка точности предварительного предсказания на паблик тесте
ansd1=np.argmax(model.predict(test[['correct_text']].astype('str').values[:50000],batch_size=1024),axis=-1)
ansd2=np.argmax(model.predict(test[['correct_text']].astype('str').values[50000:100000],batch_size=1024),axis=-1)
ansd3=np.argmax(model.predict(test[['correct_text']].astype('str').values[100000:150000],batch_size=1024),axis=-1)
ansd4=np.argmax(model.predict(test[['correct_text']].astype('str').values[150000:],batch_size=1024),axis=-1)
ansl=[]
for cnt in np.vstack((ansd1,ansd2,ansd3,ansd4)):
  ansl.append(''.join(list(map(lambda x: tvvoc[x], cnt.tolist()))))
test['correct_text2']=ansl

In [ ]:
# фильтрация условно точных предсказаний
test['tag1']=1*(test['correct_text']==test['correct_text2'])

In [ ]:
# благодаря сверточной ветке модель хорошо может дообучаться на тестовых данных (примерно +5% от начальной accuracy)
# дообучение на условно точных предсказаниях на паблик тесте
X_train,y_train=test[['corrupted_text']][test['tag1']==1],tv(tf.constant(test['correct_text'].values[test['tag1']==1])).numpy()
history=model.fit(X_train,y_train,batch_size=1024,epochs=5)

In [8]:
# загрузка весов после обучения на паблик тесте 
model.load_weights('/content/drive/MyDrive/RuCode6_2/rucode_text6.h5')

In [ ]:
# предварительное предсказание и дообучение на приват тесте
ansd1=np.argmax(model.predict(testp[['corrupted_text']].astype('str').values[:50000],batch_size=1024),axis=-1)
ansd2=np.argmax(model.predict(testp[['corrupted_text']].astype('str').values[50000:],batch_size=1024),axis=-1)
ansl=[]
for cnt in np.vstack((ansd1,ansd2)):
  ansl.append(''.join(list(map(lambda x: tvvoc[x], cnt.tolist()))))
testp['correct_text']=ansl
ansd1=np.argmax(model.predict(testp[['correct_text']].astype('str').values[:50000],batch_size=1024),axis=-1)
ansd2=np.argmax(model.predict(testp[['correct_text']].astype('str').values[50000:],batch_size=1024),axis=-1)
ansl=[]
for cnt in np.vstack((ansd1,ansd2)):
  ansl.append(''.join(list(map(lambda x: tvvoc[x], cnt.tolist()))))
testp['correct_text2']=ansl
testp['tag1']=1*(testp['correct_text']==testp['correct_text2'])
X_train,y_train=testp[['corrupted_text']][testp['tag1']==1],tv(tf.constant(testp['correct_text'].values[testp['tag1']==1])).numpy()
history=model.fit(X_train,y_train,batch_size=512,epochs=5)

In [ ]:
# предварительное конечное предсказание на приват тесте
ansd1=np.argmax(model.predict(testp[['corrupted_text']].astype('str').values[:50000],batch_size=1024),axis=-1)
ansd2=np.argmax(model.predict(testp[['corrupted_text']].astype('str').values[50000:],batch_size=1024),axis=-1)
ansl=[]
for cnt in np.vstack((ansd1,ansd2)):
  ansl.append(''.join(list(map(lambda x: tvvoc[x], cnt.tolist()))))
testp['correct_text3']=ansl

In [ ]:
# конечное предсказание на приват тесте (если модель прогнать по данным дважды точность возрастает примерно на 2%, благодаря тому что модели сложно исправить два слова за один проход)
ansd1=np.argmax(model.predict(testp[['correct_text3']].astype('str').values[:50000],batch_size=1024),axis=-1)
ansd2=np.argmax(model.predict(testp[['correct_text3']].astype('str').values[50000:],batch_size=1024),axis=-1) 
ansl=[]
for cnt in np.vstack((ansd1,ansd2)):
  ansl.append(''.join(list(map(lambda x: tvvoc[x], cnt.tolist()))))
testp['correct_text4']=ansl

In [ ]:
# сохранение результата
testp['correct_text4'].to_csv('subr2p.csv',header=False,index=False)